### Collection into pandas df

In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
host_ip = '192.168.1.8'
#iniciando cliente mongo
client = MongoClient(host_ip, 27017)
#selecionando db
db = client['news_2018']
#selecionando coleções
oantagonista = db.get_collection('oantagonista')
# collection into pandas df
oantagonista = pd.DataFrame(list(oantagonista.find()))

### Updating news in collection

In [ ]:
for index, row in carta.iterrows():
    db['veja'].update_one({'_id' : row['_id']}, 
                         {'$set' : {'section' :row['section'],
                                    'arg' :row['arg'],
                                    'sen' :row['sen'],
                                    'val' :row['val'],
                                    'mod' :row['mod'],
                                    'pre' :row['pre']
                                   }
                         })

### Update news veja

In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

# iniciando cliente mongo
client = MongoClient('localhost', 27017)
# selecionando db
veja_db = client['veja']
news = client['news']
# selecionando coleções
veja_articles = veja_db['veja']
veja_comments = veja_db['vejaComments']
# updating articles
for article in veja_articles.find():
    try:
        news['veja'].insert_one(article)
    except:
        print("article duplicate " + article['_id'], end='\r')
# updating comments articles
for comment in veja_comments.find():
    try:
        news['vejaComments'].insert_one(comment)
    except:
        print("comment duplicate " + comment['_id'], end='\r')

### Collection MetaData - compute ext size and number of comments

In [ ]:
from pymongo import MongoClient
import datetime
import time
import math

IP_HOST = 'localhost'

client = MongoClient(IP_HOST, 27017)
news_2018 = client['news_2018']
news = client['news']

index = 0
total = news['oglobo'].count()

for article in news['oglobo'].find(no_cursor_timeout=True).batch_size(5):
    if not math.isnan(float(article['date'])):
        if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) == 2018:

            # compute title size
            title_size = len(str(article['title']).split())
            # compute text size
            text_size = len(str(article['text']).split())
            # compute number of comments
            number_comments = news['ogloboComments'].find({"id_article" : article['_id']}).count()

            # insert article in news_2018
            news_2018['oglobo'].insert_one(article)

             # insert comments in news_2018
            for comment in news['ogloboComments'].find({"id_article" : article['_id']}, no_cursor_timeout=True).batch_size(5):
                news_2018['ogloboComments'].insert_one(comment)

            # insert metadata
            news_2018['metaData'].insert_one({
                'url' : article['url'],
                'text_size' : text_size,
                'number_comments' : number_comments
            })
    
    # status
    index += 1
    print('progress {0:.4f} %'.format(index / total * 100), end='\r')


### Clean News - filter news 2010/2018

In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('localhost', 27017)
db = client.news
collections = ['carta_capital', 'estadao', 'folha', 'g1', 'oantagonista', 'oglobo', 'veja']

for collection in collections:
    print('\n' + collection + '\n')

    for article in db[collection].find():
        try:
            #process Article
            if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) < 2010:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})

            elif float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) > 2018:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})
        except:
            print('delete article except')
            db[collection].delete_one({'_id':article['_id']})


### Clean Comments - filter news 2010/2018

In [ ]:
from pymongo import MongoClient
import datetime

client = MongoClient('192.168.1.7', 27017)
db = client.news
collections = ['carta_capital', 'estadao', 'folha', 'g1', 'oantagonista', 'oglobo', 'veja']

for collection in collections:
    print('\n' + collection + '\n')

    for article in db[collection].find():
        try:
            #process Article
            if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) < 2010:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})

            elif float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) > 2018:
                print('delete article' + str(datetime.datetime.fromtimestamp(int(article['date']))))
                db[collection].delete_one({'_id':article['_id']})
        except:
            print('delete article except')
            db[collection].delete_one({'_id':article['_id']})


### Selec News - selecting news 2018

In [ ]:
from pymongo import MongoClient
import datetime
import time
import re

client = MongoClient('localhost', 27017)
news = client['news']
news_2018 = client['news_2018']
collections = ['folha']
PUNCTUATION = u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ%]' # define punctuation 

def processSection(section):
    if 'pol' in section or 'poder' in section:
        return 'politica'
    elif 'opini' in section or 'felipe' in section or 'colunas' in section or 'renan' in section or 'mario' in section or 'claudio' in section:
        return 'opiniao'
    elif 'esport' in section or 'copa' in section:
        return 'esporte'
    elif 'mundo' in section or 'intern' in section:
        return 'internacional'
    elif 'econ' in section or 'merca' in section:
        return 'economia'
    elif 'ci' in section or 'tec' in section:
        return 'tecnologia'
    elif 'edu' in section or 'merca' in section:
        return 'educacao'
    else:
        return 'diversos'

for collection in collections:
    # print collection
    print('\n' + collection + '\n')
    # process collection
    for article in news[collection].find():
        if float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')) == float(2018):
            try:
                article['section'] = re.sub(PUNCTUATION, ' ', str(article['section'])).lower()
                article['section'] = processSection(article['section'])
                #article['text'] = re.sub(PUNCTUATION, ' ', str(article['text'])).lower()
                #article['title'] = re.sub(PUNCTUATION, ' ', str(article['title'])).lower()
                # insert in news 2018
                news_2018[collection].insert_one(article)
            except:
                print("Error")
        print(float(datetime.datetime.fromtimestamp(int(article['date'])).strftime('%Y')))

### News Readability

In [ ]:
from readcalc import readcalc
from pymongo import MongoClient
import nltk
nltk.download('punkt')
client = MongoClient('localhost', 27017)
news = client.news_2018
collections = ['carta_capital', 'el_pais', 'estadao', 'folha', 'gazeta_do_povo', 'oantagonista', 'oglobo', 'veja']

for collection in collections:
    for article in news[collection].find(no_cursor_timeout=True).batch_size(5):
        text_readability = readcalc.ReadCalc(str(article['text']))
        news['readabilityMetrics'].insert_one({
                'url': article['url'],
                'words':len(text_readability.get_words()),
                'unique_words':text_readability.get_internal_metrics()[2],
                'sentences':len(text_readability.get_sentences()),
                'syllables':text_readability.get_internal_metrics()[4],
                '3_syllables_or_more':text_readability.get_internal_metrics()[5],
                'coleman_liau_index': text_readability.get_coleman_liau_index(),
                'smog_index':text_readability.get_smog_index(),
                'lix_index':text_readability.get_lix_index(),
                'dale_chall_score':text_readability.get_dale_chall_score(),
                'dale_chall_known_fraction':text_readability.get_dale_chall_known_fraction()
            }) 

### Comments Readability

In [1]:
# loading abbreviation dic
with open('../dics/AB_dict') as f:
    abbreviation = f.readlines()
# process dic    
abbreviation = [x.split() for x in abbreviation]
abbreviation = {line[0]: line[1] for line in abbreviation}

# loading internet_slang dic
with open('../dics/IN_dict') as f:
    internet_slang = f.readlines()
# process dic    
internet_slang = [x.split() for x in internet_slang]
internet_slang = {line[0]: ' '.join(line[1:]) for line in internet_slang}

# loading foreign_word dic
with open('../dics/ES_dict') as f:
    foreign_word = f.readlines()
# process dic    
foreign_word = [x.split() for x in foreign_word]
foreign_word = [line for line in foreign_word if len(line) > 1] # selecting valid lines
foreign_word = {line[0]: ' '.join(line[1:]) for line in foreign_word}

In [2]:
# Lexical normalization
def lexical_normalization(text):
    for k, v in abbreviation.items():
        text = str(text).replace(k,v)        
    for k, v in internet_slang.items():
        text = str(text).replace(k,v)        
    for k, v in foreign_word.items():
        text = str(text).replace(k,v)
    return text

In [3]:
from readcalc import readcalc
from pymongo import MongoClient
import nltk
nltk.download('punkt')
client = MongoClient('localhost', 27017)
db = client['mestrado']
collections = ['oantagonistaComments', 'ogloboComments', 'vejaComments']

for collection in collections:
   
    for comment in db[collection].find(no_cursor_timeout=True).batch_size(5):
        text_readability = readcalc.ReadCalc(text=lexical_normalization(comment['text']), language="pt_BR")
        
        db[collection].update_one({'_id' : comment['_id']}, 
                         {'$set' : {'words':len(text_readability.get_words()),
                                    'unique_words':text_readability.get_internal_metrics()[2],
                                    'chars':text_readability.get_internal_metrics()[0],
                                    'sentences':len(text_readability.get_sentences()),
                                    'syllables':text_readability.get_internal_metrics()[4],
                                    '3_syllables_or_more':text_readability.get_internal_metrics()[5],                                    
                                    'flesch_reading_ease': text_readability.get_flesch_reading_ease(),
                                    'flesch_kincaid_grade_level': text_readability.get_flesch_kincaid_grade_level(),
                                    'coleman_liau_index': text_readability.get_coleman_liau_index(),
                                    'gunning_fog_index': text_readability.get_gunning_fog_index(),
                                    'smog_index':text_readability.get_smog_index(),
                                    'ari_index':text_readability.get_ari_index(),
                                    'lix_index':text_readability.get_lix_index(),
                                    'dale_chall_score':text_readability.get_dale_chall_score(),
                                    'dale_chall_known_fraction':text_readability.get_dale_chall_known_fraction()
                                   }
                         })        

[nltk_data] Downloading package punkt to /home/diogo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 100
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 100
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 29
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 48
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 276
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 45
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 44
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 34
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 170
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 126
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 126
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 141
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 196
TEXT

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 23
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 64
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 204
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 106
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 109
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 155
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 349
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 85
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 95
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 157
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 123
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 142
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 113
TEX

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 168
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 56
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 221
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 307
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 41
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 359
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 77
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 6
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 57
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 6
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 46
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 6
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 49
TEXT IS NOT

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 264
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 133
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 90
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 94
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 87
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 57
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 151
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 71
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 78
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 46
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 73
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 124
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 49
TEXT IS 

...

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 113
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 185
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 37
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 194
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 25
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 189
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 194
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 166
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 131
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 163
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 96
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 194
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 147
TE

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 36
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 175
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 136
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 136
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 41
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 85
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 110
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 199
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 194
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 161
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 133
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 196
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 92
TEX

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 78
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 59
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 69
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 9
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 67
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 236
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 215
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 268
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 88
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 40
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 45
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 83
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 30
TEXT IS NO

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 109
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 93
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 105
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 187
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 190
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 55
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 109
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 66
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 23
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 197
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 174
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 12
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 115
TEXT

KeyboardInterrupt: 

### Load data on Neo4j

In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship
from IPython.display import clear_output

host_ip = '192.168.1.7'
#iniciando cliente mongo
client = MongoClient(host_ip, 27017)
#selecionando db
db = client['news_2018']
#selecionando coleções
oglobo = db.get_collection('oglobo')
# collection into pandas df
news = pd.DataFrame(list(oglobo.find()))
ogloboComments = db.get_collection('ogloboComments')
# collection into pandas df
comments = pd.DataFrame(list(ogloboComments.find()))

In [ ]:
#Conect to sandbox
neo4j_sandbox = Graph("bolt://localhost:7687", auth=("neo4j", "admin"))
#Open transaction
graph = neo4j_sandbox.begin()

#### Nodes

In [ ]:
# insert news node
for index, row in news.iterrows():
    node = Node("News",
                section = row["section"],
                sub_title = row["sub_title"],
                author = row["author"],
                title = row["title"],
                date = row["date"],
                url = row["url"],
                text = row["text"],
                id_news = row["_id"]
               )
    graph.merge(node, "News", "url")
    #Print progress
    clear_output(wait=True)
    print('Progress: {0:.4f} %'.format((index + 1) / news.shape[0] * 100))

In [ ]:
# insert comment node
for index, row in comments.iterrows():
    node = Node("Comment",
                author = row["author"],
                date = row["date"],
                text = row["text"],
                _id = row["author"] + row["text"]
               )
    graph.merge(node, "Comment", "_id")
    #Print progress
    clear_output(wait=True)
    print('Progress: {0:.4f} %'.format((index + 1) / comments.shape[0] * 100))

In [ ]:
# insert reader node
for index, row in comments.iterrows():
    node = Node("Reader",
                name = row["author"]
               )
    graph.merge(node, "Reader", "name")
    #Print progress
    clear_output(wait=True)
    print('Progress: {0:.4f} %'.format((index + 1) / comments.shape[0] * 100))

In [ ]:
#Commit the transaction
graph.commit()
#Open transaction
graph = neo4j_sandbox.begin()

#### Relationships

In [ ]:
# insert view relationship
for index, row in comments.iterrows():
    news = neo4j_sandbox.nodes.match("News", id_news = row["id_article"]).first()
    reader = neo4j_sandbox.nodes.match("Reader", name=row["author"]).first()
    rel = Relationship(reader, 'VIEW', news)
    graph.create(rel) 
    #Print progress
    clear_output(wait=True)
    print('Progress: {0:.4f} %'.format((index + 1) / comments.shape[0] * 100))

In [ ]:
# insert view relationship
for index, row in comments.iterrows():
    news = neo4j_sandbox.nodes.match("News", id_news = row["id_article"]).first()
    comment = neo4j_sandbox.nodes.match("Comment", author = row["author"]).first()
    rel = Relationship(comment, 'PUBLISHED', news)
    rel["date"] = row["date"]
    graph.create(rel)
    #Print progress
    clear_output(wait=True)
    print('Progress: {0:.4f} %'.format((index + 1) / comments.shape[0] * 100))

In [ ]:
#Commit the transaction
graph.commit()

### Format and structure articles text fields

In [ ]:
import nltk
import re
from pymongo import MongoClient
from nltk.corpus import stopwords
nltk.download('stopwords')

HOST_IP = 'localhost' # define host's ip

client = MongoClient(HOST_IP, 27017)
news = client['news_2018']
collections = ['folha', 'gazeta_do_povo', 'oglobo', 'veja']

def processSection(section):
    if 'pol' in section or 'poder' in section:
        return 'politica'
    elif 'opini' in section or 'felipe' in section or 'colunas' in section or 'renan' in section or 'mario' in section or 'claudio' in section:
        return 'opiniao'
    elif 'esport' in section or 'copa' in section:
        return 'esporte'
    elif 'mundo' in section or 'intern' in section:
        return 'internacional'
    elif 'econ' in section or 'merca' in section:
        return 'economia'
    elif 'ci' in section or 'tec' in section:
        return 'tecnologia'
    elif 'edu' in section or 'merca' in section:
        return 'educacao'
    else:
        return 'diversos'

# function for processing text
def process_text(text):
    text = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ%]', ' ', str(text)).lower()
    text = text.split() # split sentences by words
    stop_words = stopwords.words('portuguese') # load stop words
    text = [word for word in text if word not in stop_words] # Remove stopwords
    return " ".join(text)

for collection in collections:

    len_collection = db[collection].count_documents({})
    index = 0
    
    for article in news[collection].find():
        try:
            article['section'] = processSection(process_text(article['section']))
            article['text'] = process_text(article['text'])
            article['title'] = process_text(article['title'])
            article['sub_title'] = process_text(article['sub_title'])
            # update news
            news[collection].update_one({'_id' : article['_id']}, 
                                        {'$set' : {'section':article['section'],
                                                'text':article['text'],
                                                'title':article['title']                                    
                                               }
                                         })
            # print status
            index += 1
            print('Portal: {0} - Progress: {1:.4f} %'.format(collection, index / len_collection * 100), end='\r')
            
        except:
            print('Formatting error')
                

In [ ]:
from pymongo import MongoClient
import datetime
import time
import re

client = MongoClient('localhost', 27017)

mestrado = client['mestrado']
news = client['news']

collections = ['oantagonista', 'veja']

for collection in collections:
    len_collection = mestrado[collection].count_documents({})
    index = 0
    # process collection
    for article in mestrado[collection].find(no_cursor_timeout=True).batch_size(5):
        for comment in news[collection + 'Comments'].find({'id_article': article['url']}, no_cursor_timeout=True).batch_size(5):
            try:
                mestrado[collection + 'Comments'].insert_one(comment)
            except:
                pass
        index += 1
        print('Portal: {0} - Progress: {1:.4f} %'.format(collection, index / len_collection * 100), end='\r')

In [ ]:
from pymongo import MongoClient
import datetime
import time
import re

client = MongoClient('localhost', 27017)

mestrado = client['mestrado']
news = client['news']

collections = ['oglobo']

for collection in collections:
    len_collection = mestrado[collection].count_documents({})
    index = 0
    # process collection
    for article in mestrado[collection].find(no_cursor_timeout=True).batch_size(5):
        for comment in news[collection + 'Comments'].find({'id_article': article['_id']},no_cursor_timeout=True).batch_size(5):
            try:
                comment['id_article'] = article['url']
                mestrado[collection + 'Comments'].insert_one(comment)
            except:
                pass
        index += 1
        print('Portal: {0} - Progress: {1:.4f} %'.format(collection, index / len_collection * 100), end='\r')